In [ ]:
import morePVs_output as opm
import os
import en_utilities as um
import pandas as pd
import shutil

In [2]:
project='EN1_value_of_pv2'
base_path='C:\\Users\\z5044992\\Documents\\MainDATA\\DATA_EN_3\\studies'
i_path =os.path.join(base_path,project,'inputs')
o_path =os.path.join(base_path,project,'outputs')
short_name = 'value10'

In [3]:
sites = ['A','E',  'D','B', 'H', 'I', 'G', 'C',  'J', 'F' ]
tag_list = ['a208_f12_cp34',
 'a161_f7_cp38',
 'a138_f9_cp45',
 'a104_f8_cp57',
 'a52_f3_cp27',
 'a48_f4_cp09',
 'a44_f4_cp17',
 'a34_f4_cp34',
 'a26_f4_cp44',
 'a20_f5_cp37']
site_tags = dict(zip(sites, tag_list))

In [4]:
# Combine input files
df_comb = pd.DataFrame()
studies = [s for s in os.listdir(i_path) if '.csv' in s and short_name in s and 'combined' not in s]
for st in studies:
    site = um.find_between(st,'site', '_')
    study = um.find_between(st,'study_','.csv')
    print(site, study)
    op = opm.Output(base_path = base_path,
                          project = project,
                          study_name = study)
    self = op
    dfp = self.study_parameters
    dfp['site_tag']= site_tags[site]
    dfp['ns'] = dfp.index
    dfp['ns'] = dfp['ns'].apply(lambda x : site + str(x))
    dfp = dfp.set_index(['ns'])
    dfp.index.name='scenario'
    
    
    df_comb = df_comb.append(dfp)

comb = os.path.join(i_path, 'study_combined_'+short_name+'.csv')
um.df_to_csv(df_comb, comb)

A siteA_value10


FileNotFoundError: File b'C:\\Users\\z5044992\\Documents\\MainDATA\\DATA_EN_3\\studies\\EN1_value_of_pv2\\outputs\\siteA_value10\\siteA_value10_results.csv' does not exist

In [ ]:
# Combine output files 

studies = [s for s in os.listdir(i_path) if '.csv' in s and short_name in s and 'combined' not in s]
comb_dir = os.path.join(o_path,'combined_' +short_name)
if not os.path.exists(comb_dir):
                     os.makedirs(comb_dir)
        
comb_scen_dir = os.path.join(comb_dir,'scenarios')
if not os.path.exists(comb_scen_dir):
                     os.makedirs(comb_scen_dir)
dfc_comb = pd.DataFrame()
dfr_comb = pd.DataFrame()
dfs_comb = pd.DataFrame()


for st in studies:
    site = um.find_between(st,'site', '_')
    study = um.find_between(st,'study_','.csv')
    folder = os.path.join(o_path,study)
    print(site, study)
    op = opm.Output(base_path = base_path,
                          project = project,
                          study_name = study)
    self = op
    files = [f for f in os.listdir(folder) if '.csv' in f]
    fcr  = os.path.join(folder,[f for f in files if 'customer_results' in f][0])
    fstd  = os.path.join(folder, [f for f in files if 'results_std' in f][0])
    fr  = os.path.join(folder, [f for f in files if not ('customer_results' in f or 'results_std' in f)][0])
    
    dfc=pd.read_csv(fcr)
    dfs=pd.read_csv(fstd)
    dfr=pd.read_csv(fr) 

    dfc['ns'] = dfc.scenario
    dfc['ns'] = dfc['ns'].apply(lambda x : site + str(x))
    dfc = dfc.set_index(['ns'])    
    dfc_comb = dfc_comb.append(dfc)
    dfc.index.name='scenario'
    
    dfr['ns'] = dfr.scenario
    dfr['ns'] = dfr['ns'].apply(lambda x : site + str(x))
    dfr = dfr.set_index(['ns'])    
    dfr_comb = dfr_comb.append(dfr)
    dfr.index.name='scenario'
    
    dfs['ns'] = dfs.scenario
    dfs['ns'] = dfs['ns'].apply(lambda x : site + str(x))
    dfs = dfs.set_index(['ns'])    
    dfs_comb = dfs_comb.append(dfs)
    dfs.index.name='scenario'
    
    scen_dir = os.path.join(folder,'scenarios')

    for file in os.listdir(scen_dir):
        old = os.path.join(scen_dir,file)
        new = os.path.join(comb_scen_dir,file)
        shutil.copy(old,new)
        
dfr_comb = dfr_comb.drop('scenario', axis=1)
dfr_comb.index.name='scenario'
dfc_comb = dfc_comb.drop('scenario', axis=1)
dfc_comb.index.name='scenario'
dfs_comb = dfs_comb.drop('scenario', axis=1)
dfs_comb.index.name='scenario'
        
cr = os.path.join(comb_dir, 'combined_' + short_name + '_results.csv')
um.df_to_csv(dfr_comb, cr)

cc = os.path.join(comb_dir, 'combined_' + short_name + '_customer_results.csv')
um.df_to_csv(dfc_comb, cc)

cs = os.path.join(comb_dir, 'combined_' + short_name + '_std_dev.csv')
um.df_to_csv(dfs_comb, cs)
print("**Complete**")